## Hedging Strategy on Historical Market Data with Rolling Voatilities##

In [5]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.stats import norm
import datetime as datetime
import yfinance as yf

#Import helper functions for MC/RQMC methods, GBM path generation, and asian option payoffs
import asian, paths, lds

### **Naive Application of Hedging Strategy on Historical Data**

As done in class for **European call options**, we investigated a **naive application** of the proposed **hedging strategy for Asian options** using historical market data, obtained via the `yfinance` library.

- **Period considered:** from *10/01/2023* to *10/01/2025*. 
- **Volatility estimation:** used **rolling volatilities** (21 days window) to price options and compute $\Delta_{t_i}$.  
- **GBM model**: at each time-step we assume GBM model for pricing and estimating deltas with current volatility.
- **Interest rate:** set to $r = 0$ for simplicity.


In [6]:
# Import data
tickers = [
    'AAPL', 'TSLA', 'F', 'HD', 'GM', 'SPY', '^GSPC', 'VTI', 'QQQ',
    'BA', 'UPS', 'UNP',
    'WMT', 'COST', 'PG', 'DIS', 'NKE', 'KO',
    'IWM', 'DIA', 'XLK', 'XLV', 'EFA', 'EEM', 'HYG']

stock_data = yf.download(tickers, interval='1d', start='2023-10-01', end='2025-10-11')

# Rolling vols and mask (start at index 21 so we have 21-day history)
for ticker in tickers:
    stock_data['log_returns', ticker] = np.log(stock_data['Close', ticker].pct_change() + 1.0)
    stock_data['volatility',  ticker] = stock_data['log_returns', ticker].rolling(window=21).std() * np.sqrt(252.0)
mask = stock_data.index[21:]

# Parameters
r        = 0.00                 # interest rate
n_paths  = 2**13                # RQMC paths for pricing/Δ0
R        = 16                   # RQMC replicates
rel_bump = 1e-4                 # CFD bump for arithmetic Δ0 only
base_seed = 123

/var/folders/_3/7y11b1d17n7_ss5ry15n418h0000gn/T/ipykernel_48033/3541137097.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(tickers, interval='1d', start='2023-10-01', end='2025-10-11')
[*********************100%***********************]  25 of 25 completed


In [7]:
# GBM paths, payoffs, variance reduction tecniques functions from imported files
def _precompute_bridge(T_local, d_local):
    return paths.precompute_all(T_local, d_local)

def _build_paths_from_Z(Z, S0_local, r, sigma_local, T_local, d_local, precomp):
    return paths.generate_paths(method='bridge', S0=S0_local, r=r, sigma=sigma_local,
                                T=T_local, d=d_local, Z=Z, precomp=precomp)

def _payoff_arith(S_paths, K, r, T):
    return asian.asian_payoff_paths(S_paths, K=K, r=r, T=T, option="call", average='arith')

def _payoff_geom(S_paths, K, r, T):
    return asian.asian_payoff_paths(S_paths, K=K, r=r, T=T, option="call", average='geom')

# Geometric control variate estimator
def _cv_adjust(x_samples, y_samples, Ey):
    y = y_samples - y_samples.mean()
    x = x_samples - x_samples.mean()
    var_y = np.mean(y*y)
    b = 0.0 if var_y <= 0 else np.mean(x*y)/var_y
    return x_samples - b*(y_samples - Ey), b

# Antithetic variate estimator
def _antithetic_payoffs_S0(Z, S0_local, r, sigma_local, T_local, d_local, precomp, K):
    S_plus  = _build_paths_from_Z(Z,   S0_local, r, sigma_local, T_local, d_local, precomp)
    S_minus = _build_paths_from_Z(-Z,  S0_local, r, sigma_local, T_local, d_local, precomp)
    A_bar = 0.5*(_payoff_arith(S_plus, K, r, T_local) + _payoff_arith(S_minus, K, r, T_local))
    G_bar = 0.5*(_payoff_geom (S_plus, K, r, T_local) + _payoff_geom (S_minus, K, r, T_local))
    return A_bar, G_bar

# RQMC + geometric control variate and antithetic variate for pricing in arithemtic case
def price_rqmc_geomcv_generic(S0_local, K, r, sigma_local, T_local, d_local,
                              n_paths, R_repl, base_seed=0):
    base_U = lds.sobol_points(n=n_paths, d=d_local, scramble=False, seed=base_seed)
    means, bstars = [], []
    precomp = _precompute_bridge(T_local, d_local)
    GEOM_CF = asian.geometric_asian_call_price_discrete(S0=S0_local, K=K, r=r, sigma=sigma_local,
                                                        T=T_local, d=d_local)
    for rr in range(R_repl):
        U = lds.random_digital_shift(base_U, seed=base_seed + 1000 + rr)
        Z = lds.u_to_normal(U)
        A_bar, G_bar = _antithetic_payoffs_S0(Z, S0_local, r, sigma_local, T_local, d_local, precomp, K)
        adj, b = _cv_adjust(A_bar, G_bar, GEOM_CF)
        means.append(float(adj.mean()))
        bstars.append(b)
    means = np.array(means, float)
    return float(means.mean()), (float(means.std(ddof=1)) if R_repl>1 else 0.0), float(np.nanmean(bstars))

# Delta0 for arithmetic cases via central finite differences on estimates by RQMC + geometric control variate and antithetic variate for pricing in arithemtic case
def delta0_cfd_rqmc_geomcv_generic(S0_local, K, r, sigma_local, T_local, d_local,
                                   n_paths, R_repl, h_rel=1e-4, base_seed=0):
    h = h_rel * S0_local
    base_U = lds.sobol_points(n=n_paths, d=d_local, scramble=False, seed=base_seed)
    estimates = []
    precomp = _precompute_bridge(T_local, d_local)
    for rr in range(R_repl):
        U = lds.random_digital_shift(base_U, seed=base_seed + 2000 + rr)
        Z = lds.u_to_normal(U)
        vals = []
        for S0b in (S0_local + h, S0_local - h):
            A_bar, G_bar = _antithetic_payoffs_S0(Z, S0b, r, sigma_local, T_local, d_local, precomp, K)
            GEOM_CF_b = asian.geometric_asian_call_price_discrete(S0=S0b, K=K, r=r, sigma=sigma_local,
                                                                  T=T_local, d=d_local)
            adj, _ = _cv_adjust(A_bar, G_bar, GEOM_CF_b)
            vals.append(float(adj.mean()))
        Vp, Vm = vals
        estimates.append((Vp - Vm) / (2*h))
    estimates = np.array(estimates, float)
    return float(estimates.mean()), (float(estimates.std(ddof=1)) if R_repl>1 else 0.0)

# Closed-form geometric pieces with clipping and rolling volatility:

EPS_S  = 1e-12 # Floor value to avoid zeros
MIN_SIG, MAX_SIG = 1e-6, 3.0     # clip volatilities for stability [0.0001%, 300%]

def clip_sigma(sig):
    sig = float(sig) if np.isfinite(sig) else np.nan
    if not np.isfinite(sig):
        return None
    return float(np.clip(sig, MIN_SIG, MAX_SIG))

def _geom_log_stats_future_uniform_safe(Si, i, r, sigma_i, T_local, d_local):
    n = d_local
    n_rem = n - (i + 1)
    if n_rem <= 0:
        return 0.0, 0.0
    dt  = T_local / n
    tau = np.arange(1, n_rem + 1) * dt
    mu_L  = (n_rem / n) * np.log(max(Si, EPS_S)) + ((r - 0.5*sigma_i**2) / n) * np.sum(tau)
    k = np.arange(1, n_rem + 1, dtype=float)
    sum_min = dt * np.sum(k*(k+1)/2.0 + (n_rem - k)*k)
    var_L = (sigma_i**2 / (n**2)) * sum_min
    return mu_L, var_L

def geom_delta_remaining_conditional(Si, i, known_fixings, sigma_i, r, T_local, d_local, K):
    sig = clip_sigma(sigma_i)
    if sig is None:
        return 0.0

    n = d_local
    n_rem = n - (i + 1)
    if n_rem <= 0:
        return 0.0

    log_c_known = (1.0/n) * np.sum(np.log(np.maximum(np.asarray(known_fixings, float), EPS_S))) if (i+1)>0 else 0.0

    mu_L, var_L = _geom_log_stats_future_uniform_safe(Si, i, r, sig, T_local, d_local)
    mu = log_c_known + mu_L
    s2 = max(var_L, 0.0)
    s  = float(np.sqrt(s2))

    a = n_rem / n
    if s == 0.0:
        G = np.exp(mu)
        return (a / max(Si, EPS_S)) * (G if (G > K) else 0.0)

    EG = np.exp(mu + 0.5*s2)
    d1 = (mu - np.log(max(K, EPS_S)) + s2) / s
    return (a / max(Si, EPS_S)) * EG * norm.cdf(d1)

def geom_delta_0_closed_form_discrete(S0, K, r, sigma, T, d):
    S0 = float(max(S0, EPS_S))
    mu = np.log(S0) + (r - 0.5*sigma**2) * T * (d + 1) / (2.0 * d)
    s2 = sigma**2 * T * (d + 1) * (2.0*d + 1.0) / (6.0 * d**2)
    s  = np.sqrt(max(s2, 0.0))
    EG = np.exp(mu + 0.5*s2)
    d1 = (mu - np.log(max(K, EPS_S)) + s2) / s if s > 0 else (np.inf if np.exp(mu) > K else -np.inf)
    return np.exp(-r*T) * (EG * norm.cdf(d1)) / S0


# Hedging strategy on data:

def hedge_one_real_path_predictable(S_series, vol_series, r, rel_bump):
    """
    Inputs:
      S_series: array of closes at d days
      vol_series: array of rolling volatilities corresponding to S_series
    Returns:
    V0_arith, PV_unhedged_arith, PV_hedged_arith V0_geom,  PV_unhedged_geom,  PV_hedged_geom)
    """
    d_local = int(len(S_series))
    if d_local < 3:
        return (np.nan,)*6
    T_local = d_local / 252.0
    S0_local = float(S_series[0])
    K_local  = S0_local

    # choose initial volatility with clipping if necessary
    sigma0 = None
    for s in vol_series:
        cs = clip_sigma(s)
        if cs is not None:
            sigma0 = cs
            break
    if sigma0 is None:
        return (np.nan,)*6

    # Pricing at initial time
    # Arithmetic via RQMC + geometric CV
    V0_arith, _, _ = price_rqmc_geomcv_generic(S0_local, K_local, r, sigma0, T_local, d_local,
                                               n_paths=n_paths, R_repl=R, base_seed=base_seed+1)
    # Arithmetic Delta0 via CFD
    Delta0_arith, _ = delta0_cfd_rqmc_geomcv_generic(S0_local, K_local, r, sigma0, T_local, d_local,
                                                     n_paths=n_paths, R_repl=R, h_rel=rel_bump, base_seed=base_seed+2)
    # Geometric price and Delta0 closed-form
    V0_geom  = asian.geometric_asian_call_price_discrete(S0=S0_local, K=K_local, r=r,
                                                         sigma=sigma0, T=T_local, d=d_local)
    Delta0_geom = geom_delta_0_closed_form_discrete(S0_local, K_local, r, sigma0, T_local, d_local)

    # Realized discounted payoffs
    disc = np.exp(-r*T_local)
    payoff_arith_disc = disc * max(float(np.mean(S_series)) - K_local, 0.0)
    # geometric mean of observed prices
    payoff_geom_disc  = disc * max(float(np.exp(np.mean(np.log(np.maximum(S_series, _EPS_S))))) - K_local, 0.0)

    # Hedging increments of S along realized path
    t_grid = np.arange(1, d_local+1) * (T_local / d_local)
    pv_disc = np.exp(-r * t_grid)
    pvS = pv_disc * S_series
    pvS_shift = np.concatenate(([S0_local], pvS))
    dpvS = pvS_shift[1:] - pvS_shift[:-1]  # length d_local

    # Initialize deltas for each interval (t_i, t_{i+1}]
    dlt_arith = np.zeros(d_local, float)
    dlt_geom  = np.zeros(d_local, float)

    # Delta for first interval:
    dlt_arith[0] = Delta0_arith     # arithmetic uses CFD 
    dlt_geom[0]  = Delta0_geom      # geometric uses closed-form

    known_fixings = []              # realized part of the path known up to t_i

    for i in range(1, d_local):
        # After observing S_{t_i}, use it to set hedge for next interval
        known_fixings.append(S_series[i-1])

        sig_i = clip_sigma(vol_series[i-1]) or sigma0  # volatility to use for delta chosen at t_i

        # Conditional geometric delta (shares) based on info up to t_i
        d_next = geom_delta_remaining_conditional(S_series[i-1], i-1, known_fixings,
                                                                  sig_i, r, T_local, d_local, K_local)
        dlt_arith[i] = d_next   # proxy for arithmetic
        dlt_geom[i]  = d_next   # exact hedge for geometric under given volatility for GBM

    # Profits (seller's point of view)
    pv_unhedged_arith = V0_arith - payoff_arith_disc
    pv_hedged_arith   = V0_arith + float(np.dot(dlt_arith, dpvS)) - payoff_arith_disc

    pv_unhedged_geom  = V0_geom  - payoff_geom_disc
    pv_hedged_geom    = V0_geom  + float(np.dot(dlt_geom,  dpvS)) - payoff_geom_disc

    return V0_arith, pv_unhedged_arith, pv_hedged_arith, V0_geom, pv_unhedged_geom, pv_hedged_geom


### **Discussion**

Although real-world market data does **not fully satisfy the assumptions of the Black–Scholes model**,  
the implemented **hedging strategy** still had a **significant impact**, mitigating and **avoiding the large losses** observed with the unhedged strategy, for **both arithmetic and geometric case**.


In [8]:
# Run experiment on retrieved data
rows = []
for ticker in tickers:
    try:
        S_ser   = stock_data['Close', ticker].loc[mask].astype(float).values
        vol_ser = stock_data['volatility', ticker].loc[mask].astype(float).values
    except Exception:
        rows.append((ticker, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan))
        continue

    valid = np.isfinite(S_ser) & np.isfinite(vol_ser)
    S_ser, vol_ser = S_ser[valid], vol_ser[valid]
    if len(S_ser) < 30:
        rows.append((ticker, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan))
        continue

    (V0_arith, unhedged_arith, hedged_arith,
     V0_geom,  unhedged_geom,  hedged_geom) = hedge_one_real_path_predictable(S_ser, vol_ser, r=r, rel_bump=rel_bump)

    rows.append((ticker,
                 V0_arith, unhedged_arith, hedged_arith,
                 V0_geom,  unhedged_geom,  hedged_geom))

results_df = pd.DataFrame(
    rows,
    columns=[
        'Ticker',
        'Estimated price (Arithmetic)',
        'Unhedged profit (Arithmetic)',
        'Hedged profit (Arithmetic)',
        'Closed form price (Geometric)',
        'Unhedged profit (Geometric)',
        'Hedged profit (Geometric)'
    ]
)

display(results_df)


,Ticker,Estimated price (Arithmetic),Unhedged profit (Arithmetic),Hedged profit (Arithmetic),Closed form price (Geometric),Unhedged profit (Geometric),Hedged profit (Geometric)
0,AAPL,8.676932,-32.588839,-4.810199,8.319130,-31.564284,-3.785644
1,TSLA,32.633448,-37.936391,-17.242443,28.112894,-31.353564,-10.659616
2,F,1.322963,-0.535868,0.521312,1.149854,-0.664169,0.393011
3,HD,16.249189,-74.715630,-2.464118,15.462808,-74.003337,-1.751826
4,GM,2.871196,-15.740512,-0.196710,2.624381,-15.328089,0.215712
5,SPY,18.754984,-125.648215,1.779625,18.063382,-123.087189,4.340651
6,^GSPC,194.576958,-1213.184831,23.110127,187.339490,-1191.153822,45.141135
7,VTI,9.629577,-61.713585,0.857157,9.260000,-60.521934,2.048808
8,QQQ,21.247813,-113.805172,2.347090,20.195522,-111.448921,4.703342
9,BA,14.975396,11.977883,-1.591923,13.995976,12.994070,-0.575736
